In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import artm

# change log style
lc = artm.messages.ConfigureLoggingArgs()
lc.minloglevel = 3
lib = artm.wrapper.LibArtm(logging_config=lc)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
from copy import deepcopy
from topicnet.cooking_machine.models.topic_model import TopicModel
from topicnet.cooking_machine.cubes import RegularizersModifierCube
from topicnet.cooking_machine.experiment import Experiment
from topicnet.cooking_machine.cubes import *
from topicnet.cooking_machine.dataset import Dataset

%load_ext autoreload
%autoreload 2

In [3]:
import topicnet.cooking_machine.cubes as tncubes

In [4]:
from IPython.core.display import display, HTML
from IPython.display import clear_output, display_html
display(HTML("""<style>
.container { width:90% !important; }
div.output_scroll .output_subarea { white-space: pre; }
</style>"""))

In [5]:
! rm -r topicnet/experiments/

## Инициализация модели

Создаем `ARTM` модель:

In [6]:
DATA_PATH_SUB = 'topicnet/subtitles.csv'
DATA_PATH_SCI = 'topicnet/PScience.csv'

In [7]:
dataset_sub = Dataset(DATA_PATH_SUB)
dictionary_sub = dataset_sub.get_dictionary()

dataset_sci = Dataset(DATA_PATH_SCI)
dictionary_sci = dataset_sci.get_dictionary()

In [8]:
dataset_sci.get_possible_modalities()

{'@2gramm', '@3gramm', '@author', '@post_tag', '@snippet', '@title', '@word'}

In [9]:
from topicnet.cooking_machine.baselines import init_simple_default_model

n_topics = 15
n_background = 1
model_artm = init_simple_default_model(
    dictionary=dictionary_sci,
    modalities_to_use=dataset_sci.get_possible_modalities(),
    main_modality='@word',
    n_specific_topics=n_topics - n_background,
    n_background_topics=n_background,
)
specific_topics, bacground_topics = model_artm.topic_names[:-n_background], model_artm.topic_names[-n_background:]
model_artm.scores

[PerplexityScore@all, SparsityThetaScore, SparsityPhiScore@2gramm, PerplexityScore@2gramm, TopicKernel@2gramm, SparsityPhiScore@post_tag, PerplexityScore@post_tag, TopicKernel@post_tag, SparsityPhiScore@3gramm, PerplexityScore@3gramm, TopicKernel@3gramm, SparsityPhiScore@word, PerplexityScore@word, TopicKernel@word, SparsityPhiScore@snippet, PerplexityScore@snippet, TopicKernel@snippet, SparsityPhiScore@author, PerplexityScore@author, TopicKernel@author, SparsityPhiScore@title, PerplexityScore@title, TopicKernel@title]

## Cooking Machine

Создаем объект `TopicModel` и `Experiment`:
##### Для `TopicModel` можно объявить кастомный скор
Скор наследуется от базового класса отвечающего за хранение и обновление истории метрики

In [10]:
from topicnet.cooking_machine.models.base_score import BaseScore

class ThatCustomScore(BaseScore):
    def __init__(self):
        super().__init__()

    def call(self, model,
             score_to_max = 'TopicKernel@word',
             score_to_min='TopicKernel@word',
             topic='background_14'):
        want_higher = model.get_score('TopicKernel@word').average_kernel_contrast
        want_lower = model.get_score('TopicKernel@word').kernel_contrast[-1]
        
        return want_higher - want_lower

In [11]:
custom_scores = {'GoodinessMeasure': ThatCustomScore()}
tm = TopicModel(model_artm, model_id='new_id', custom_scores=custom_scores)

In [12]:
# experiment starts without model

experiment = Experiment(experiment_id="Test_", save_path="topicnet/experiments")

#experiment = Experiment(experiment_id="Test_", save_path="topicnet/experiments", model=tm)

#### Воспроизведём стратегию Насти Яниной из статьи "Мультимодальные тематические модели для разведочного поиска в коллективном блоге"
Для этого понадобится описание стратегии через параметры кубов добавляющих регуляризаторы и перебирающих значения их гиперпараметров

In [13]:
from topicnet.cooking_machine.cubes.perplexity_strategy import retrieve_score_for_strategy
from topicnet.cooking_machine.cubes.perplexity_strategy import PerplexityStrategy

cube_iters = 5

cube_settings = [
    {'CubeCreator':{
        'model': tm,
        'num_iter': cube_iters,
        'parameters':[
            {'name': 'seed',
             'values': [82019, 322],
            },
        ],
        'reg_search': 'grid',
    },
    'selection': [
        f'model.seed = 82019 and PerplexityScore@all -> min',
        f'model.seed = 322 and PerplexityScore@all -> min']
    },
    {'RegularizersModifierCube':{
        'num_iter': cube_iters,
        'strategy': PerplexityStrategy(0.0, 10, 15),
        'tracked_score_function': retrieve_score_for_strategy('GoodinessMeasure'),
        'regularizer_parameters': {
             "regularizer": artm.SmoothSparsePhiRegularizer(
                 name='smooth_phi_bcg',
                 topic_names=bacground_topics,
                 tau=1,
                 class_ids=['@word']),
             "tau_grid": [],
            },
        'reg_search': 'add',
        'verbose': False,
    },
    'selection': [f'model.seed = 82019 and GoodinessMeasure -> max',
                  f'model.seed = 322 and GoodinessMeasure -> max']        
    },
    {'RegularizersModifierCube':{
        'num_iter': cube_iters,
        'strategy': PerplexityStrategy(0.0, 10, 15),
        'tracked_score_function': retrieve_score_for_strategy('GoodinessMeasure'),
        'regularizer_parameters': {
             "regularizer": artm.SmoothSparseThetaRegularizer(
                 name='smooth_theta_bcg',
                 topic_names=bacground_topics,
                 tau=1),
             "tau_grid": [],
            },
        'reg_search': 'add',
        'verbose': False,
    },
    'selection': [f'model.seed = 82019 and GoodinessMeasure -> max',
                  f'model.seed = 322 and GoodinessMeasure -> max']
    },
]
class_ids=['@word', '@2gramm', '@3gramm']
for modality in class_ids:
    cube_settings += [{
        'RegularizersModifierCube':{
            'num_iter': cube_iters,
            'strategy': PerplexityStrategy(1, 10, 10),
            'tracked_score_function': retrieve_score_for_strategy(f'PerplexityScore{modality}'),
            'regularizer_parameters': {
             "regularizer": artm.DecorrelatorPhiRegularizer(
                 name=f'decorrelation_phi_{modality}',
                 topic_names=specific_topics,
                 tau=1,
                 class_ids=[modality]),
             'tau_grid': [],
            },
            'reg_search': 'mul',
            'verbose': True,
        },
        'selection': [f'model.seed = 82019 and PerplexityScore{modality} -> min',
                      f'model.seed = 322 and PerplexityScore{modality} -> min']
    }]
    cube_settings += [{
        'RegularizersModifierCube':{
            'num_iter': cube_iters,
            'strategy': PerplexityStrategy(0.0, -10, 15),
            'tracked_score_function': retrieve_score_for_strategy(f'PerplexityScore{modality}'),
            'regularizer_parameters': {
                "regularizer": artm.SmoothSparsePhiRegularizer(
                     name=f'sparse_phi_{modality}',
                     topic_names=specific_topics,
                     tau=1,
                     class_ids=[modality]),
                'tau_grid': [],
            },
            'reg_search': 'add',
            'verbose': False,
        },
        'selection': [f'model.seed = 82019 and PerplexityScore{modality} < 1.1 * MINIMUM(PerplexityScore{modality}) and SparsityPhiScore{modality} -> max',
                      f'model.seed = 322 and PerplexityScore{modality} < 1.1 * MINIMUM(PerplexityScore{modality}) and SparsityPhiScore{modality} -> max']
    }]
cube_settings += [{
    'RegularizersModifierCube':{
        'num_iter': cube_iters,
        'strategy': PerplexityStrategy(0.0, 10, 15),
        'tracked_score_function': retrieve_score_for_strategy(f'SparsityThetaScore'),
        'regularizer_parameters': {
            "regularizer": artm.SmoothSparseThetaRegularizer(
                 name='smooth_theta',
                 topic_names=specific_topics,
                 tau=1),
            'tau_grid': [],
        },
        'reg_search': 'add',
        'verbose': True,
    },
    'selection': [f'model.seed = 82019 and PerplexityScore{modality} < 1.1 * MINIMUM(PerplexityScore{modality}) and SparsityThetaScore -> max',
                  f'model.seed = 322 and PerplexityScore{modality} < 1.1 * MINIMUM(PerplexityScore{modality}) and SparsityThetaScore -> max']
}]

In [14]:
experiment.build(cube_settings)

In [15]:
display(HTML("""<style>
div .output_subarea > pre {
  white-space: pre;
  word-wrap: normal;
}
</style>"""))

In [16]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', RuntimeWarning)
    final_models = experiment.run(dataset_sci, verbose=True, nb_verbose=True)

Experiment Test_

Experiment was made with BigARTM 0.10.0
Tree:
                                                        ┌21h33m49s_07d08m2019y
                       ┌21h33m29s_07d08m2019y───────────┤
                       │                                ├21h34m01s_07d08m2019y
                       │                                │                                  ┌21h35m13s_07d08m2019y
                       │                                │                                  ├21h35m26s_07d08m2019y
                       │                                │                                  │                                  ┌21h44m18s_07d08m2019y
                       │                                │                                  │                                  ├21h44m35s_07d08m2019y
                       │                                │                                  │                                  ├21h44m51s_07d08m2019y
                       │                     

In [17]:
tm.info

config {
  topic_name: "topic_0"
  topic_name: "topic_1"
  topic_name: "topic_2"
  topic_name: "topic_3"
  topic_name: "topic_4"
  topic_name: "topic_5"
  topic_name: "topic_6"
  topic_name: "topic_7"
  topic_name: "topic_8"
  topic_name: "topic_9"
  topic_name: "topic_10"
  topic_name: "topic_11"
  topic_name: "topic_12"
  topic_name: "topic_13"
  topic_name: "background_14"
  class_id: "@2gramm"
  class_id: "@post_tag"
  class_id: "@3gramm"
  class_id: "@word"
  class_id: "@snippet"
  class_id: "@author"
  class_id: "@title"
  class_weight: 8.780309677124023
  class_weight: 79.51461029052734
  class_weight: 245.16552734375
  class_weight: 1.0
  class_weight: 31.14565658569336
  class_weight: 389.0713806152344
  class_weight: 105.8414306640625
  score_config {
    name: "PerplexityScore@all"
    type: ScoreType_Perplexity
    config: "\010\001\022$d99a5640-f468-4bbf-b93a-5abe512d0cea\032\007@2gramm\032\t@post_tag\032\007@3gramm\032\005@word\032\010@snippet\032\007@author\032\006@title

In [18]:
display(HTML("""<style>
.container { width:90% !important; }
div.output_scroll .output_subarea { white-space: pre; }
</style>"""))

Посмотрим на перплексию полученных во время тренировки моделей

In [19]:
from matplotlib import cm

tms = experiment.models
n=len(tms)
colors = np.linspace(0, 0.65, n)

plt.figure(figsize=(8,8))
for i, tm in enumerate(list(tms.values())[2:]):
    perp_score = tm.scores['PerplexityScore@all']
    plt.plot(perp_score[1:], color=cm.Blues(colors[i]))
plt.show()

KeyError: 'PerplexityScore'

<Figure size 576x576 with 0 Axes>

In [ ]:
len(tms)

## Давайте посмотрим на отобранные модели

Сначала посмотрим на топ токены тем получившихся моделей

In [20]:
num_show = 2
show_all = False
n = None if show_all else num_show 
first_model = final_models[0]
second_model = final_models[1]

In [21]:
from topicnet.viewers.top_tokens_viewer import TopTokensViewer

In [22]:
thresh = 1e-5
first_model_top_tok = TopTokensViewer(first_model, num_top_tokens=10, method='phi')
second_model_top_tok = TopTokensViewer(second_model, num_top_tokens=10, method='phi')

first_model_html =  first_model_top_tok.to_html(first_model_top_tok.view(),thresh=thresh)
second_model_html = second_model_top_tok.to_html(second_model_top_tok.view(),thresh=thresh)

Методы вывода топ токенов: 
* 'top/phi'
* 'blei'
* 'pwt'
* 'likelihood'
* 'tfidf' (special)

In [23]:
for line in first_model_html[:n]:
    display_html(line, raw=True)

,topic_0 @2gramm
центральный_банк,0.0098
нейронный_сеть,0.00788
государственный_долг,0.00618
центральный_банка,0.00591
средний_класс,0.00479
развитый_страна,0.0045
среднее_класс,0.00437
крупный_город,0.00412
машинный_обучение,0.00392
рак_грудь,0.00373


,topic_1 @2gramm
теория_струна,0.00965
магнитный_поль,0.00802
чёрный_дыра,0.00801
квантовый_компьютер,0.0079
длина_волна,0.00729
квантовый_механика,0.00657
скорость_свет,0.00631
магнитный_пол,0.0062
электромагнитный_волна,0.00619
закон_физика,0.00472


In [24]:
for line in second_model_html[:n]:
    display_html(line, raw=True)

,topic_0 @2gramm
массовый_культура,0.01233
xx_век,0.00606
речь_идти,0.00532
культовый_кино,0.00486
cinema_studies,0.00474
крайний_мера,0.00401
языковой_семья,0.0039
советский_союз,0.0038
общество_потребление,0.00367
индоевропейский_язык,0.00355


,topic_1 @2gramm
нобелевский_премия,0.01
квантовый_компьютер,0.00772
длина_волна,0.00551
раковый_клетка,0.00479
органический_химия,0.00438
твёрдый_тело,0.00421
углеродный_нанотрубка,0.0042
квантовый_точка,0.00414
квантовый_технология,0.00397
магнитный_поль,0.00388


In [25]:
from topicnet.viewers.top_documents_viewer import TopDocumentsViewer

In [26]:
first_model_top_doc = TopDocumentsViewer(first_model, dataset=dataset_sci).view()
second_model_top_doc = TopDocumentsViewer(second_model, dataset=dataset_sci).view()

In [27]:
for i, topic_docs in enumerate(first_model_top_doc[:n]):
    print('topic_'+str(i))
    topic_docs = [str(topic) for topic in topic_docs]
    for doc_id in topic_docs:
        print(doc_id,' ',dataset_sci.get_source_document(document_id=doc_id))
    print('='*100)

topic_0
823.txt   ['@title Как взаимодействуют фискальная и монетарная политика? | @snippet Экономист Сергей Пекарский о макроэкономическом анализе, борьбе с инфляцией и контроле обменного курса\n']
3140.txt   ['@title Создана нейронная сеть, работающая по принципу машины Тьюринга | @snippet Об архитектуре фон Неймана, машине Тьюринга и проверке работоспособности\n']
3111.txt   ['@title FAQ: Финансовая репрессия | @snippet 8 фактов о механизмах регулирования государством финансовых рынков в развивающихся экономиках\n']
224.txt   ['@title FAQ: Гендерная идентичность и адаптация | @snippet 6 фактов о мужских и женских социальных признаках, стратегиях поведения и хромосомных аномалиях\n']
1960.txt   ['@title Взаимодействие фискальной и монетарной политики | @snippet Экономист Сергей Пекарский о политике стимулирования экспорта, курсе валют и методах борьбы с инфляцией\n']
665.txt   ['@title Когда мы станем бессмертными? | @snippet Биофизик Петр Федичев о диагностике инфекционных заболеван

In [28]:
for i, topic_docs in enumerate(first_model_top_doc[:n]):
    print('topic_'+str(i))
    topic_docs = [str(topic) for topic in topic_docs]
    for doc_id in topic_docs:
        print(doc_id,' ',dataset_sci.get_source_document(document_id=doc_id))
    print('='*100)

topic_0
823.txt   ['@title Как взаимодействуют фискальная и монетарная политика? | @snippet Экономист Сергей Пекарский о макроэкономическом анализе, борьбе с инфляцией и контроле обменного курса\n']
3140.txt   ['@title Создана нейронная сеть, работающая по принципу машины Тьюринга | @snippet Об архитектуре фон Неймана, машине Тьюринга и проверке работоспособности\n']
3111.txt   ['@title FAQ: Финансовая репрессия | @snippet 8 фактов о механизмах регулирования государством финансовых рынков в развивающихся экономиках\n']
224.txt   ['@title FAQ: Гендерная идентичность и адаптация | @snippet 6 фактов о мужских и женских социальных признаках, стратегиях поведения и хромосомных аномалиях\n']
1960.txt   ['@title Взаимодействие фискальной и монетарной политики | @snippet Экономист Сергей Пекарский о политике стимулирования экспорта, курсе валют и методах борьбы с инфляцией\n']
665.txt   ['@title Когда мы станем бессмертными? | @snippet Биофизик Петр Федичев о диагностике инфекционных заболеван

Давайте посмотрим насколько похожи темы моделей друг на друга

In [29]:
from topicnet.viewers.spectrum import TopicSpectrumViewer

In [30]:
first_spectrum = TopicSpectrumViewer(first_model, early_stopping=1000, class_ids=['@word']).view()
second_spectrum = TopicSpectrumViewer(second_model, early_stopping=1000, class_ids=['@word']).view()

spectrum_order_first = first_model_top_tok.to_html(first_model_top_tok.view(), first_spectrum, thresh=thresh)
spectrum_order_second = second_model_top_tok.to_html(second_model_top_tok.view(), second_spectrum, thresh=thresh,)

In [31]:
for line in spectrum_order_first[:n]:
    display_html(line, raw=True)

,topic_0 @2gramm
центральный_банк,0.0098
нейронный_сеть,0.00788
государственный_долг,0.00618
центральный_банка,0.00591
средний_класс,0.00479
развитый_страна,0.0045
среднее_класс,0.00437
крупный_город,0.00412
машинный_обучение,0.00392
рак_грудь,0.00373


,topic_2 @2gramm
крайний_мера,0.00609
нобелевский_премия,0.00475
нитрид_бора,0.0043
экономический_рост,0.00386
массовый_культура,0.00378
рабочий_место,0.00358
современный_общество,0.00344
повседневный_жизнь,0.00322
речь_идти,0.00315
общество_потребление,0.00306


In [32]:
for line in spectrum_order_second[:n]:
    display_html(line, raw=True)

,topic_0 @2gramm
массовый_культура,0.01233
xx_век,0.00606
речь_идти,0.00532
культовый_кино,0.00486
cinema_studies,0.00474
крайний_мера,0.00401
языковой_семья,0.0039
советский_союз,0.0038
общество_потребление,0.00367
индоевропейский_язык,0.00355


,topic_6 @2gramm
xx_век,0.00932
русский_язык,0.00867
xix_век,0.00581
критический_теория,0.00527
речь_идти,0.005
история_наука,0.00363
гуманитарный_наука,0.00354
мировой_война,0.00323
хх_век,0.0032
крайний_мера,0.00314


Давайте сравним темы двух моделей

In [33]:
from topicnet.viewers.topic_mapping import TopicMapViewer

In [34]:
close_themes = TopicMapViewer(model=first_model, second_model=second_model).view()

In [35]:
map_first = first_model_top_tok.to_html(first_model_top_tok.view(), close_themes[0], thresh=thresh)
map_second = second_model_top_tok.to_html(second_model_top_tok.view(), close_themes[1], thresh=thresh)

for line_first, line_second in zip(map_first[:n], map_second[:n]):
    display_html(line_first, raw=True)
    display_html(line_second, raw=True)
    print('='*100)

,topic_0 @2gramm
центральный_банк,0.0098
нейронный_сеть,0.00788
государственный_долг,0.00618
центральный_банка,0.00591
средний_класс,0.00479
развитый_страна,0.0045
среднее_класс,0.00437
крупный_город,0.00412
машинный_обучение,0.00392
рак_грудь,0.00373


,topic_12 @2gramm
экономический_рост,0.00425
принятие_решение,0.00341
северный_кавказ,0.00331
крайний_мера,0.00323
европейский_союз,0.00304
конец_конец,0.00292
речь_идти,0.00263
саудовский_аравия,0.00246
рынок_труд,0.00241
городской_пространство,0.00238


,topic_1 @2gramm
теория_струна,0.00965
магнитный_поль,0.00802
чёрный_дыра,0.00801
квантовый_компьютер,0.0079
длина_волна,0.00729
квантовый_механика,0.00657
скорость_свет,0.00631
магнитный_пол,0.0062
электромагнитный_волна,0.00619
закон_физика,0.00472


,topic_13 @2gramm
чёрный_дыра,0.02319
социальный_сеть,0.00851
гравитационный_волна,0.00532
теория_относительность,0.0045
система_отсчёт,0.00446
электромагнитный_волна,0.00435
трансгенный_растение,0.004
большой_количество,0.00393
анализ_дать,0.00393
скорость_свет,0.00382
